#Conjoint Analysis Exercise


##What is Conjoint Analysis

Conjoint Analysis, short for "consider jointly" is a marketing insight technique that provides consumers with combinations, pairs or groups of products that are a combination of various features and ask them what they prefer. 

## Full-profile Conjoing Analysis
Full-profile Conjoint Analysis  is one of the most fundamental approaches for measuring attribute utilities. In a full-profile conjoint task, different product descriptions are developed, ranked and presented to the consumer for preference evaluations.

##Setting up

In [0]:
import pandas as pd
import numpy as np
import statsmodels.api as sm
from statsmodels.stats.outliers_influence import variance_inflation_factor    

In [2]:
from google.colab import files
uploaded = files.upload()

Saving MARK 5826 - Rank the Bike you would buy (Responses) - Sheet3.csv to MARK 5826 - Rank the Bike you would buy (Responses) - Sheet3.csv


In [0]:
df = pd.read_csv('MARK 5826 - Rank the Bike you would buy (Responses) - Sheet3.csv')


In [5]:
df.head()

,Unnamed: 0,Unnamed: 1,Unnamed: 2,Unnamed: 3,Unnamed: 4,Unnamed: 5,Unnamed: 6,Unnamed: 7,Unnamed: 8,Unnamed: 9,Unnamed: 10,Unnamed: 11,Unnamed: 12,Unnamed: 13,Unnamed: 14,Unnamed: 15,Unnamed: 16,Unnamed: 17,Unnamed: 18,Unnamed: 19,Unnamed: 20,Unnamed: 21,Unnamed: 22,Unnamed: 23,Unnamed: 24,Unnamed: 25,Unnamed: 26,Unnamed: 27,Unnamed: 28,Unnamed: 29,Unnamed: 30,Unnamed: 31,Unnamed: 32,Unnamed: 33
0,Have you ever biked before?,Yes,Yes,Yes,Yes,Yes,Yes,Yes,Yes,Yes,Yes,Yes,No,Yes,Yes,Yes,Yes,Yes,No,Yes,Yes,No,Yes,Yes,Yes,Yes,Yes,Yes,Yes,Yes,No,Yes,Yes,Yes
1,Gender,Male,Female,Female,Female,Male,Female,Male,Female,Female,Female,Male,Male,Female,Female,Male,Male,Male,Female,Male,Male,Female,Female,Male,Female,Female,Female,Female,Female,Female,Female,Male,Female,Female
2,"Bike Preference [Mountain bike, Has gears, $1000]",8,1,1,1,1,7,8,6,7,6,5,5,7,2,1,2,8,4,4,8,1,1,7,3,3,5,4,1,2,7,7,4,5
3,"Bike Preference [Mountain bike, No gears, $1000]",7,2,2,2,3,8,6,7,8,1,2,3,8,1,2,1,7,3,3,7,3,2,1,1,1,6,3,2,1,6,2,3,1
4,"Bike Preference [Mountain bike, Has gears, $500]",6,3,5,4,2,1,5,8,6,8,7,6,5,6,3,4,6,6,8,6,6,8,8,5,8,7,8,5,6,8,8,2,7


##Part 1: Dataframe

Creating Dataframe for python to understand your data.


In [0]:
df_col = ['Rank','Gender','Experience','Type','Gear','Price']
df1 = pd.DataFrame(columns = df_col)
col_df = df.columns

In [7]:
df1

,Rank,Gender,Experience,Type,Gear,Price


In [0]:
def bike_type_gear_price(tmp):
  brac_start = tmp.find('[')
  brac_end = tmp.find(']')

  bike = (tmp[brac_start+1:brac_end])
  bike = bike.split(",")
  bike[2] = bike[2].replace('$','')
  return bike

In [0]:
# df['Unnamed: 0'][2:len(df['Unnamed: 0'])]

2    Bike Preference [Mountain bike, Has gears, $1000]
3     Bike Preference [Mountain bike, No gears, $1000]
4     Bike Preference [Mountain bike, Has gears, $500]
5      Bike Preference [Mountain bike, No gears, $500]
6      Bike Preference [Racing bike, Has gears, $1000]
7       Bike Preference [Racing bike, No gears, $1000]
8       Bike Preference [Racing bike, Has gears, $500]
9        Bike Preference [Racing bike, No gears, $500]
Name: Unnamed: 0, dtype: object

In [0]:
#df1 = df1.iloc[0:0]
#df1

In [0]:
#rank = Unnamed: 1	[2:9]
#gender = Unnamed: 1	[1]
#exp = Unnamed: 1	[0]
col_start =1 
for col_start in range (1,len(col_df)):
  for start in range(2,10):
    tgp = bike_type_gear_price(df['Unnamed: 0'][start])
    col_name = col_df[col_start]
    new_data = [df[col_name][start], df[col_name][1], df[col_name][0], tgp[0], tgp[1], tgp[2]]
    df1_row = len(df1)
    df1.loc[df1_row] = new_data

In [10]:
df1.head()

,Rank,Gender,Experience,Type,Gear,Price
0,8,Male,Yes,Mountain bike,Has gears,1000
1,7,Male,Yes,Mountain bike,No gears,1000
2,6,Male,Yes,Mountain bike,Has gears,500
3,5,Male,Yes,Mountain bike,No gears,500
4,4,Male,Yes,Racing bike,Has gears,1000


##Part 2: Dummy Variables

Computers cannot understand words properly. Hence, we change every variables to dummy variables (0,1) in order for computer to understand.

In [11]:
conjoint_data = pd.get_dummies(df1, columns = ['Price','Gender','Experience','Type','Gear'])
conjoint_data.head()

,Rank,Price_ 1000,Price_ 500,Gender_Female,Gender_Male,Experience_No,Experience_Yes,Type_Mountain bike,Type_Racing bike,Gear_ Has gears,Gear_ No gears
0,8,1,0,0,1,0,1,1,0,1,0
1,7,1,0,0,1,0,1,1,0,0,1
2,6,0,1,0,1,0,1,1,0,1,0
3,5,0,1,0,1,0,1,1,0,0,1
4,4,1,0,0,1,0,1,0,1,1,0


In [0]:
newColName = {"Rank": "Rank", "Price":"Price", "Gender_Female": "Female", "Gender_Male":"Male",
             "Experience_No": "No exp", "Experience_Yes": "Yes Exp", "Type_Mountain bike": "Mountain Bike",
             "Type_Racing bike": "Racing Bike", "Gear_ Has gears": "Geared", "Gear_ No gears": "No Gear"}
conjoint_data.rename(columns=newColName, inplace = True)

In [0]:
conjoint_data.pop("No exp") 
conjoint_data.pop("Racing Bike") 
conjoint_data.pop("No Gear") 
conjoint_data.pop("Female") 
conjoint_data.pop("Price_ 500")

In [14]:
sorted_data = conjoint_data.sort_values(by=['Rank'], ascending=False)
sorted_data.head(15)

,Rank,Price_ 1000,Male,Yes Exp,Mountain Bike,Geared
0,8,1,1,1,1,1
39,8,0,1,1,0,0
178,8,0,1,1,1,1
222,8,0,0,1,0,1
94,8,0,1,0,0,1
30,8,0,0,1,0,1
234,8,0,0,0,1,1
190,8,0,0,1,0,1
194,8,0,0,1,1,1
86,8,0,1,1,0,1


In [0]:
newColName = {"Rank": "Rank", "Price_ 1000":"Price", "Male": "Gender",
             "Yes Exp": "Exp", "Mountain Bike": "Bike Type",
             "Geared": "Gear"}
conjoint_data.rename(columns=newColName, inplace = True)

- Price: 1 = 1000, 0 = 500
- Gender: 1 = male, 0 = female
- Exp: 1 = Yes , 0 = No
- Bike Type: 1 = Mountain Bike , 0 = Racing Bike
- Gear: 1 = Geared, 0 = No Gear

In [16]:
conjoint_data.head()

,Rank,Price,Gender,Exp,Bike Type,Gear
0,8,1,1,1,1,1
1,7,1,1,1,1,0
2,6,0,1,1,1,1
3,5,0,1,1,1,0
4,4,1,1,1,0,1


## Part 3: Customer Segmentation

In [0]:
male = conjoint_data["Gender"] == 1
female = conjoint_data["Gender"] == 0
yes_exp = conjoint_data["Exp"] == 1
no_exp = conjoint_data["Exp"] == 0


In [0]:
male

In [0]:
male = conjoint_data[male]
female = conjoint_data[female] 
yes_exp = conjoint_data[yes_exp]
no_exp = conjoint_data[no_exp]

In [20]:
male.head()

,Rank,Price,Gender,Exp,Bike Type,Gear
0,8,1,1,1,1,1
1,7,1,1,1,1,0
2,6,0,1,1,1,1
3,5,0,1,1,1,0
4,4,1,1,1,0,1


In [21]:
everything = conjoint_data[["Price", "Gender","Exp", "Bike Type", "Gear"]]
everything = sm.add_constant(everything)
Y = conjoint_data.pop("Rank")

/usr/local/lib/python2.7/dist-packages/numpy/core/fromnumeric.py:2389: FutureWarning: Method .ptp is deprecated and will be removed in a future version. Use numpy.ptp instead.
  return ptp(axis=axis, out=out, **kwargs)


In [23]:
everything = conjoint_data[["Price", "Gender","Exp", "Bike Type", "Gear"]]
everything.head()

,Price,Gender,Exp,Bike Type,Gear
0,1,1,1,1,1
1,1,1,1,1,0
2,0,1,1,1,1
3,0,1,1,1,0
4,1,1,1,0,1


# Ordinary Linear Regression (OLS)


![alt text](https://upload.wikimedia.org/wikipedia/commons/thumb/3/3a/Linear_regression.svg/1200px-Linear_regression.svg.png)

#Everything OLS

In [25]:
linearRegression = sm.OLS(Y.astype(float) ,everything.astype(float)).fit()
linearRegression.summary()

<class 'statsmodels.iolib.summary.Summary'>
"""
                                 OLS Regression Results                                
=======================================================================================
Dep. Variable:                   Rank   R-squared (uncentered):                   0.748
Model:                            OLS   Adj. R-squared (uncentered):              0.743
Method:                 Least Squares   F-statistic:                              153.4
Date:                Thu, 27 Jun 2019   Prob (F-statistic):                    2.77e-75
Time:                        01:04:28   Log-Likelihood:                         -620.40
No. Observations:                 264   AIC:                                      1251.
Df Residuals:                     259   BIC:                                      1269.
Df Model:                           5                                                  
Covariance Type:            nonrobust                                                  
==============================================================================
                 coef    std err          t      P>|t|      [0.025      0.975]
------------------------------------------------------------------------------
Price         -0.3111      0.301     -1.033      0.303      -0.904       0.282
Gender         0.4245      0.325      1.306      0.193      -0.216       1.065
Exp            3.1617      0.305     10.352      0.000       2.560       3.763
Bike Type      0.8555      0.301      2.839      0.005       0.262       1.449
Gear           1.8101      0.301      6.007      0.000       1.217       2.403
==============================================================================
Omnibus:                        2.687   Durbin-Watson:                   1.314
Prob(Omnibus):                  0.261   Jarque-Bera (JB):                2.178
Skew:                           0.090   Prob(JB):                        0.337
Kurtosis:                       2.593   Cond. No.                         3.38
==============================================================================

Warnings:
[1] Standard Errors assume that the covariance matrix of the errors is correctly specified.
"""

#Male OLS

In [31]:
male.head()

,Price,Gender,Exp,Bike Type,Gear
0,1,1,1,1,1
1,1,1,1,1,0
2,0,1,1,1,1
3,0,1,1,1,0
4,1,1,1,0,1


In [28]:
male_x = male[["Price","Bike Type", "Gear"]]
male_x = sm.add_constant(male_x)
male_y = male.pop("Rank")
linearRegression = sm.OLS(male_y.astype(float) ,male_x.astype(float)).fit()
linearRegression.summary()

KeyError: ignored

#Female OLS

In [0]:
female_x = female[["Price","Bike Type", "Gear"]]
female_x = sm.add_constant(female_x)

female_y = female.pop("Rank")
linearRegression = sm.OLS(female_y.astype(float) ,female_x.astype(float)).fit()
linearRegression.summary()

<class 'statsmodels.iolib.summary.Summary'>
"""
                            OLS Regression Results                            
==============================================================================
Dep. Variable:                   Rank   R-squared:                       0.127
Model:                            OLS   Adj. R-squared:                  0.112
Method:                 Least Squares   F-statistic:                     7.986
Date:                Thu, 27 Jun 2019   Prob (F-statistic):           5.31e-05
Time:                        00:20:26   Log-Likelihood:                -366.22
No. Observations:                 168   AIC:                             740.4
Df Residuals:                     164   BIC:                             752.9
Df Model:                           3                                         
Covariance Type:            nonrobust                                         
==============================================================================
                 coef    std err          t      P>|t|      [0.025      0.975]
------------------------------------------------------------------------------
const          4.8333      0.334     14.460      0.000       4.173       5.493
Price         -1.4048      0.334     -4.203      0.000      -2.065      -0.745
Bike Type     -0.0952      0.334     -0.285      0.776      -0.755       0.565
Gear           0.8333      0.334      2.493      0.014       0.173       1.493
==============================================================================
Omnibus:                       11.819   Durbin-Watson:                   1.604
Prob(Omnibus):                  0.003   Jarque-Bera (JB):                4.617
Skew:                           0.002   Prob(JB):                       0.0994
Kurtosis:                       2.188   Cond. No.                         3.73
==============================================================================

Warnings:
[1] Standard Errors assume that the covariance matrix of the errors is correctly specified.
"""

#yes_Exp OLS

In [32]:
yes_exp_x = yes_exp[["Price","Bike Type", "Gear"]]
yes_exp_x = sm.add_constant(yes_exp_x)

yes_exp_y = yes_exp.pop("Rank")
linearRegression = sm.OLS(yes_exp_y.astype(float) ,yes_exp_x.astype(float)).fit()
linearRegression.summary()

KeyError: ignored

#No Exp OLS

In [0]:
no_exp_x = no_exp[["Price","Bike Type", "Gear"]]
no_exp_x = sm.add_constant(no_exp_x)
no_exp_y = no_exp.pop("Rank")
linearRegression = sm.OLS(no_exp_y.astype(float) ,no_exp_x.astype(float)).fit()
linearRegression.summary()

<class 'statsmodels.iolib.summary.Summary'>
"""
                            OLS Regression Results                            
==============================================================================
Dep. Variable:                   Rank   R-squared:                       0.222
Model:                            OLS   Adj. R-squared:                  0.139
Method:                 Least Squares   F-statistic:                     2.670
Date:                Thu, 27 Jun 2019   Prob (F-statistic):             0.0668
Time:                        00:20:32   Log-Likelihood:                -67.912
No. Observations:                  32   AIC:                             143.8
Df Residuals:                      28   BIC:                             149.7
Df Model:                           3                                         
Covariance Type:            nonrobust                                         
==============================================================================
                 coef    std err          t      P>|t|      [0.025      0.975]
------------------------------------------------------------------------------
const          3.8125      0.764      4.993      0.000       2.248       5.377
Price         -1.1250      0.764     -1.473      0.152      -2.689       0.439
Bike Type      0.8750      0.764      1.146      0.262      -0.689       2.439
Gear           1.6250      0.764      2.128      0.042       0.061       3.189
==============================================================================
Omnibus:                        0.717   Durbin-Watson:                   1.581
Prob(Omnibus):                  0.699   Jarque-Bera (JB):                0.795
Skew:                           0.249   Prob(JB):                        0.672
Kurtosis:                       2.411   Cond. No.                         3.73
==============================================================================

Warnings:
[1] Standard Errors assume that the covariance matrix of the errors is correctly specified.
"""

![alt text](https://wpforms.com/wp-content/uploads/2017/04/thank-you-page-examples.jpg)